In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer


c:\Users\alber\Desktop\projetos\Generative-IA\venv-genai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
checkpoint = "bigscience/bloomz-560m"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

In [3]:
inputs = tokenizer.encode("Translate to Portuguese: Hello.", return_tensors="pt")

In [4]:
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

Translate to Portuguese: Hello. Olá.</s>



# Fine-tuning (PEFT LoRA) para gerar SQL em Português — CPU-friendly

Este trecho adiciona um pipeline **Hugging Face Transformers + PEFT (LoRA)** para treinar um **modelo causal pequeno** a gerar **consultas SQL** a partir de **perguntas em português**.  
Projeto pensado para **CPU** (vai ser lento, mas funciona) e com **dataset simples** para prova de conceito.

> **Modelo base sugerido:** `facebook/opt-350m` (~350M de parâmetros).  
> Você pode trocar por outro causal LM compatível.


In [ ]:

# (Opcional) Instale as dependências quando rodar localmente
# Remova o comentário se precisar instalar
# !pip install -U transformers datasets peft accelerate


In [6]:
import os
from dataclasses import dataclass
from typing import Dict, List

import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    set_seed,
)
from transformers.data.data_collator import default_data_collator
from peft import LoraConfig, get_peft_model

In [7]:

# ====== Configurações principais ======
MODEL_NAME = "facebook/opt-350m"  # ~350M params, roda em CPU (lento)
OUTPUT_DIR = "./opt350m-lora-sql-pt"
MAX_SEQ_LEN = 384
SEED = 42

# CPU-only
device = torch.device("cpu")
torch.set_num_threads(max(1, os.cpu_count() // 2))  # limite para não travar a máquina
set_seed(SEED)


In [8]:

# Defina um esquema simples para orientar o modelo
SCHEMA = """
Tabelas e colunas (dialeto: PostgreSQL):

clientes(id, nome, cidade)
pedidos(id, cliente_id, data, total)
itens_pedido(id, pedido_id, produto_id, quantidade, preco)
produtos(id, nome, categoria)
"""

# Dataset mínimo: NL em PT -> SQL alvo
raw_samples: List[Dict[str, str]] = [
    {
        "pergunta": "Liste o nome dos clientes e suas cidades.",
        "sql": "SELECT nome, cidade FROM clientes;"
    },
    {
        "pergunta": "Conte quantos clientes existem por cidade.",
        "sql": "SELECT cidade, COUNT(*) AS total_clientes FROM clientes GROUP BY cidade ORDER BY total_clientes DESC;"
    },
    {
        "pergunta": "Quais pedidos têm valor total acima de 1000? Retorne id e total.",
        "sql": "SELECT id, total FROM pedidos WHERE total > 1000;"
    },
    {
        "pergunta": "Liste o total de vendas por mês (YYYY-MM).",
        "sql": "SELECT TO_CHAR(data, 'YYYY-MM') AS mes, SUM(total) AS total_vendas FROM pedidos GROUP BY mes ORDER BY mes;"
    },
    {
        "pergunta": "Quais são os 5 produtos mais vendidos em quantidade?",
        "sql": "SELECT p.nome, SUM(i.quantidade) AS qtd FROM itens_pedido i JOIN produtos p ON p.id = i.produto_id GROUP BY p.nome ORDER BY qtd DESC LIMIT 5;"
    },
    {
        "pergunta": "Qual é o faturamento total por categoria de produto?",
        "sql": "SELECT p.categoria, SUM(i.quantidade * i.preco) AS faturamento FROM itens_pedido i JOIN produtos p ON p.id = i.produto_id GROUP BY p.categoria ORDER BY faturamento DESC;"
    },
    {
        "pergunta": "Liste os pedidos feitos por clientes da cidade de São Paulo.",
        "sql": "SELECT pe.* FROM pedidos pe JOIN clientes c ON c.id = pe.cliente_id WHERE c.cidade = 'São Paulo';"
    },
    {
        "pergunta": "Para cada cliente, quantos pedidos foram realizados?",
        "sql": "SELECT c.nome, COUNT(pe.id) AS num_pedidos FROM clientes c LEFT JOIN pedidos pe ON pe.cliente_id = c.id GROUP BY c.nome ORDER BY num_pedidos DESC;"
    },
    {
        "pergunta": "Liste os produtos que nunca foram vendidos.",
        "sql": "SELECT p.* FROM produtos p LEFT JOIN itens_pedido i ON i.produto_id = p.id WHERE i.id IS NULL;"
    },
    {
        "pergunta": "Qual é o ticket médio por pedido?",
        "sql": "SELECT AVG(total) AS ticket_medio FROM pedidos;"
    },
    {
        "pergunta": "Liste os pedidos e o nome do cliente correspondente.",
        "sql": "SELECT pe.id, pe.data, pe.total, c.nome FROM pedidos pe JOIN clientes c ON c.id = pe.cliente_id;"
    },
    {
        "pergunta": "Qual é a receita total do sistema?",
        "sql": "SELECT SUM(total) AS receita_total FROM pedidos;"
    },
    {
        "pergunta": "Mostre o top 3 clientes com maior valor total em pedidos.",
        "sql": "SELECT c.nome, SUM(pe.total) AS gasto FROM clientes c JOIN pedidos pe ON pe.cliente_id = c.id GROUP BY c.nome ORDER BY gasto DESC LIMIT 3;"
    },
    {
        "pergunta": "Quais produtos pertencem à categoria 'Eletrônicos'?",
        "sql": "SELECT * FROM produtos WHERE categoria = 'Eletrônicos';"
    },
    {
        "pergunta": "Liste os pedidos feitos no ano de 2024.",
        "sql": "SELECT * FROM pedidos WHERE EXTRACT(YEAR FROM data) = 2024;"
    },
    {
        "pergunta": "Para cada mês, quantos pedidos foram realizados?",
        "sql": "SELECT TO_CHAR(data, 'YYYY-MM') AS mes, COUNT(*) AS num_pedidos FROM pedidos GROUP BY mes ORDER BY mes;"
    },
    {
        "pergunta": "Qual cliente realizou mais pedidos?",
        "sql": "SELECT c.nome, COUNT(pe.id) AS num_pedidos FROM clientes c JOIN pedidos pe ON pe.cliente_id = c.id GROUP BY c.nome ORDER BY num_pedidos DESC LIMIT 1;"
    },
    {
        "pergunta": "Quais produtos tiveram faturamento acima de 5000?",
        "sql": "SELECT p.nome, SUM(i.quantidade * i.preco) AS faturamento FROM itens_pedido i JOIN produtos p ON p.id = i.produto_id GROUP BY p.nome HAVING SUM(i.quantidade * i.preco) > 5000 ORDER BY faturamento DESC;"
    },
    {
        "pergunta": "Liste o total de itens vendidos por produto no ano de 2023.",
        "sql": "SELECT p.nome, SUM(i.quantidade) AS total_itens FROM itens_pedido i JOIN produtos p ON p.id = i.produto_id JOIN pedidos pe ON pe.id = i.pedido_id WHERE EXTRACT(YEAR FROM pe.data) = 2023 GROUP BY p.nome ORDER BY total_itens DESC;"
    },
    {
        "pergunta": "Quais clientes nunca realizaram pedidos?",
        "sql": "SELECT c.* FROM clientes c LEFT JOIN pedidos pe ON pe.cliente_id = c.id WHERE pe.id IS NULL;"
    },
]

# Formatação estilo "instrução" + "resposta" para causal LM
INSTRUCTION_TMPL = """### Tarefa
Escreva a consulta SQL em {dialeto} correspondente à pergunta.

### Esquema
{schema}

### Pergunta
{pergunta}

### Consulta SQL"""

In [9]:
def format_example(ex: Dict[str, str]) -> Dict[str, str]:
    prompt = INSTRUCTION_TMPL.format(dialeto="PostgreSQL", schema=SCHEMA.strip(), pergunta=ex["pergunta"].strip())
    target = ex["sql"].strip()
    return {
        "prompt": prompt,
        "target": target,
        "text": prompt + "\n" + target  # útil para debug
    }

dataset = Dataset.from_list([format_example(s) for s in raw_samples])
# Para demo simples, vamos usar todo mundo como treino
ds = DatasetDict({"train": dataset})
len(ds["train"])


20

In [10]:

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

c:\Users\alber\Desktop\projetos\Generative-IA\venv-genai\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\alber\.cache\huggingface\hub\models--facebook--opt-350m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [11]:
def tokenize_and_mask(batch):
    # tokeniza prompt e prompt+target para mascarar corretamente
    prompts = batch["prompt"]
    targets = batch["target"]
    input_ids_list = []
    labels_list = []
    attention_masks = []

    for p, t in zip(prompts, targets):
        # tokens do prompt (sem o SQL)
        tok_prompt = tokenizer(
            p,
            truncation=True,
            max_length=MAX_SEQ_LEN,
            padding=False,
            add_special_tokens=True,
        )

        # tokens do prompt + target (SQL)
        full_txt = p + "\n" + t
        tok_full = tokenizer(
            full_txt,
            truncation=True,
            max_length=MAX_SEQ_LEN,
            padding="max_length",
            add_special_tokens=True,
        )

        input_ids = tok_full["input_ids"]
        attention_mask = tok_full["attention_mask"]

        # Máscara: tudo que pertence ao prompt vira -100; o restante (SQL) é treinado
        labels = input_ids.copy()
        prompt_len = len(tok_prompt["input_ids"])
        for i in range(min(prompt_len, len(labels))):
            labels[i] = -100

        input_ids_list.append(input_ids)
        labels_list.append(labels)
        attention_masks.append(attention_mask)

    return {
        "input_ids": input_ids_list,
        "labels": labels_list,
        "attention_mask": attention_masks,
    }

tokenized = ds["train"].map(tokenize_and_mask, batched=True, remove_columns=["prompt","target","text"])
tokenized = tokenized.shuffle(seed=SEED)
tokenized[0].keys()


Map: 100%|██████████| 20/20 [00:00<00:00, 361.69 examples/s]


dict_keys(['input_ids', 'labels', 'attention_mask'])

In [12]:

# Carrega modelo base em CPU
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,
)
model.to(device)



Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=409

In [13]:
# Configura LoRA (atenção do OPT usa q_proj/v_proj)
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"],
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 786,432 || all params: 331,982,848 || trainable%: 0.2369


In [14]:

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,  # simula batch 16 em CPU
    learning_rate=2e-4,
    num_train_epochs=1,              # para demo; aumente em casos reais
    warmup_ratio=0.1,
    logging_steps=5,
    save_steps=50,
    save_total_limit=1,
    report_to=[],
    fp16=False,
    bf16=False,
)


In [15]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=default_data_collator,
)

trainer.train()

c:\Users\alber\Desktop\projetos\Generative-IA\venv-genai\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


RuntimeError: 
            Some tensors share memory, this will lead to duplicate memory on disk and potential differences when loading them again: [{'base_model.model.lm_head.weight', 'base_model.model.model.decoder.embed_tokens.weight'}].
            A potential way to correctly save your model is to use `save_model`.
            More information at https://huggingface.co/docs/safetensors/torch_shared_tensors
            

In [ ]:

# Salva somente adapters LoRA
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("Treino concluído. Adapters LoRA salvos em:", OUTPUT_DIR)

In [ ]:

from peft import PeftModel

# Recarrega base + adapters (CPU)
base = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float32).to(device)
base = PeftModel.from_pretrained(base, OUTPUT_DIR).to(device)
base.eval()

GEN_CFG = dict(max_new_tokens=128, temperature=0.2, top_p=0.9, do_sample=True)

def gerar_sql(pergunta: str, dialeto: str = "PostgreSQL") -> str:
    prompt = f"""### Tarefa
Escreva a consulta SQL em {dialeto} correspondente à pergunta.

### Esquema
{SCHEMA.strip()}

### Pergunta
{pergunta}

### Consulta SQL"""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=MAX_SEQ_LEN).to(device)
    with torch.no_grad():
        out = base.generate(**inputs, **GEN_CFG, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(out[0], skip_special_tokens=True)




In [ ]:
# Teste rápido
print(gerar_sql("Qual é o ticket médio por pedido?"))